In [1]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import col, expr

# Spark oturumu
builder = SparkSession.builder \
    .appName("Gold Layer - CLTV") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# RFM tablosunu oku
rfm = spark.read.format("delta").load("../delta/gold/rfm_table")

# CLTV hesapla
rfm_cltv = rfm.withColumn("CLTV", (col("Frequency") * col("Monetary")) / (col("Recency") + expr("1")))

# Sonuçları kaydet
rfm_cltv.write.format("delta").mode("overwrite").save("../delta/gold/cltv_table")

rfm_cltv.show(5)

:: loading settings :: url = jar:file:/opt/miniconda3/envs/spark-delta-env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
25/07/28 15:03:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Ivy Default Cache set to: /Users/alialtunoglu/.ivy2/cache
The jars for the packages stored in: /Users/alialtunoglu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f568732e-b06e-4052-a0dd-26215eb40ca2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 76ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

25/07/28 15:04:00 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------+---------+-----------------+------------------+
|CustomerID|Recency|Frequency|         Monetary|              CLTV|
+----------+-------+---------+-----------------+------------------+
|   15039.0|      9|     1502|19914.43999999999| 2991148.887999999|
|   13259.0|     61|       87|292.3199999999999| 410.1909677419353|
|   16982.0|     60|       22|           384.06| 138.5134426229508|
|   17966.0|     37|       68|          1098.43|1965.6115789473686|
|   13178.0|     26|      265|5725.469999999999| 56194.42777777777|
+----------+-------+---------+-----------------+------------------+
only showing top 5 rows

